# Asemic writing with Flat

By [Allison Parrish](https://www.decontextualize.com/)

This is a tutorial on how to use [flat](https://xxyxyz.org/flat), the random functions from [Numpy](http://www.numpy.org/) and [bezmerizing](https://github.com/aparrish/bezmerizing/) (a small library I made with simple functions for working with Bézier curves) to make various kinds of asemic writing.

## Installation and preliminaries

There are a number of Python libraries for drawing vector graphics, but [flat](https://xxyxyz.org/flat) is my favorite one to work with. It's small, powerful and elegant. It's also written in pure Python (i.e., does not need compiled code) and has no external dependencies, which makes it especially easy to install:

In [1]:
!pip install flat

/bin/bash: pip: comando não encontrado


In this tutorial, I'm only using a subset of flat's functionality. The cell below imports the three functions from the library that I'm going to use.

In [2]:
from flat import document, shape, rgba

The flat library can export images in a handful of formats, including PNG. We can leverage this to "embed" the results of running flat code right into a Jupyter Notebook as we're working. The cell below creates a function `show()` that takes a flat `page` object, renders it as an SVG, and displays it in the notebook. I use this extensively below!

In [3]:
from IPython.display import SVG, display
def show(page):
    display(SVG(page.svg()))

[Bezmerizing](https://github.com/aparrish/bezmerizing/) is a small library I made with a handful of useful functions for working with Bézier curves, including a function for making smooth curved paths based on a list of points, and a function that can style such paths by adjusting their thickness along the path. Install it directly from GitHub like so:

In [4]:
!pip install https://github.com/aparrish/bezmerizing/archive/master.zip

/bin/bash: pip: comando não encontrado


In [5]:
from bezmerizing import Polyline

Assuming you're running this notebook with Anaconda, you probably already have Numpy installed. [Numpy](http://www.numpy.org/) is a widely-used Python library for math and statistics. I'm going to show you how to use a handful of functions from Numpy for generating random numbers. Use the cells below to import them into your notebook:

In [6]:
import numpy as np

In [7]:
from numpy.random import uniform, normal, choice

## Making shapes with flat

Drawing with flat consists of a few steps:

1. Create a document
2. Add a page to the document
3. Create shapes
4. Place shapes on the page
5. Display the page (or the document).

A minimal example skips steps 3 and 4:

In [8]:
d = document(100, 100, 'mm') # create a document 100mm x 100mm (can replace 'mm' with 'pt')
page = d.addpage() # add a page
show(page) # show the page

... but doesn't display anything. We've just made a blank page.

To add a shape, create a `shape` object with the `shape()` function. The object returned from this function has methods that let you set the fill, stroke, and stroke width of the shape. Colors are specified with the `rgba()` function, which takes four parameters: the red, green, and blue components of the color (in the range 0—255), along with an alpha value (which controls how "transparent" the color will be—0 is completely transparent and 255 is completely opaque).

In [9]:
figure = shape()
figure.fill(rgba(255, 128, 255, 255))
figure.stroke(rgba(128, 0, 255, 255))
figure.width(4)

A `shape` object acts as a kind of container for a list of paths to draw with the fill and stroke you specified when you created the object. Add a rectangle to the shape with the `.rectangle()` method:

In [10]:
rect = figure.rectangle(5, 5, 90, 90)

This creates a rectangle at position (5, 5) on the page with a width of 90 and a height of 90. ([See the documentation for a full list of shape methods and an explanation of what their parameters mean.](https://xxyxyz.org/flat))

> Note for first-time programmers: Any word to the *left* of an equal sign (`=`) is called a *variable*. A "variable" takes the result of running some code and stores it for later under the name that you give it—sort of naming a file or putting a label on a folder. I used the word `rect` above to store the result of creating the `rectangle` shape, but you can use whatever word you want, as long as you use the same word when referring back to that value later. (There's nothing magical about calling it `rect`—I could have called is `lawrence` or `afkjsdhf`. Using `rect` just makes it easier to remember what I was using that variable for.

To make this rectangle appear on the page, pass the value returned from the method to the `.place()` method of the page:

In [11]:
d = document(100, 100, 'mm')
page = d.addpage()
page.place(rect)
show(page)

Nice! You'll notice that flat's coordinate system is set up such that the *upper left-hand corner* of the page is (0, 0). The X-axis increases as you move to the right of the page, while the Y-axes increases as you move *downwards*. (This is the opposite of how it works in a regular Cartesian coordinate system, but it's very common in computer graphics for some reason.)

Because it's so common to set the fill, stroke and stroke width at once, flat supports a "chained" syntax so that you can set them all one one line. It looks like this:

In [12]:
figure = shape().fill(rgba(255, 128, 255, 255)).stroke(rgba(128, 0, 255, 255)).width(4)

With all of this in mind, I put in the cell below all of the code necessary to draw the rectangle, from the creation of the document to displaying the page. Most of the rest of the examples in this tutorial will look like this, with all of the code to display a particular shape in the same cell.

In [13]:
d = document(100, 100, 'mm')
page = d.addpage()
figure = shape().fill(rgba(255, 128, 255, 255)).stroke(rgba(128, 0, 255, 255)).width(4)
rect = figure.rectangle(5, 5, 90, 90)
page.place(rect)
show(page)

To add another shape to the drawing, add a second call to a `shape` method. Another easy one to try out is `.circle()`, which draws a circle of a particular radius centered on the given (x, y) coordinate. Note that you have to call `page.place()` with all of the components of the shape you want to appear on in the drawing.

In [14]:
d = document(100, 100, 'mm')
page = d.addpage()
figure = shape().fill(rgba(255, 128, 255, 255)).stroke(rgba(128, 0, 255, 255)).width(4)
rect = figure.rectangle(5, 5, 90, 90)
circ = figure.circle(33, 33, 25)
page.place(rect)
page.place(circ)
show(page)

If you want the circle to be a different color (or have a different stroke color or stroke width) from the rectangle, you need to create a second `shape` object:

In [15]:
d = document(100, 100, 'mm')
page = d.addpage()
# shape for rectangle
figure = shape().fill(rgba(255, 128, 255, 255)).stroke(rgba(128, 0, 255, 255)).width(4)
# shape for circle
green_figure = shape().fill(rgba(128, 240, 128, 255)).nostroke()
# add rectangle to figure
rect = figure.rectangle(5, 5, 90, 90)
# add circle to green figure
circ = green_figure.circle(33, 33, 25)
# place them both
page.place(rect)
page.place(circ)
show(page)

Note that `.nostroke()` causes the shape to not have a stroke at all. Similarly, `.nofill()` will make the shape have no fill color.

> Exercise ideas: Modify the cell above to change the coordinates and colors of the rectangle and circle. Add a new element to the drawing using a new shape that we haven't discussed yet (say, `ellipse` or `line`) using a stroke and fill different from the other shapes.

## Randomness with numpy

For the purposes of this tutorial, let's think about asemic writing as a kind of random process. Our method of generating asemic writing with a computer will be to simulate the random movement of a hand holding a pen. The movement of the pen is constrained in certain ways—i.e., at any point in the process, some pen movements are more likely to happen than other pen movements. To model this process, we need a way to generate random numbers.

Fortunately, the Numpy library has a wide variety of random number generators with different useful properties. Let's talk about a few of them.

### Uniform distribution

The `uniform()` function returns a number between 0 and 1, drawn from a "uniform distribution." The word "uniform" here means that every value in the domain has *the same chance of being generated*—no number or range of numbers is more likely than any other. Run the cell below multiple times to see what happens.

In [16]:
uniform()

0.32256808518788527

Using a `for` loop, you can generate a series of numbers to see what the distribution looks like. (You can change the number `8` to something larger to generate more numbers and better understand the distribution.)

In [17]:
for i in range(8):
    print(uniform())

0.574543600652639
0.1764854790844027
0.9139598188925209
0.22575939635599185
0.15015153002357673
0.0698822000145699
0.9549220076660901
0.3178152361732457


If you give two parameters to `uniform()`, the function will evaluate to a number chosen from that range (with the first parameter being the low end of the range, and the second parameter being the high end.)

In [18]:
for i in range(8):
    print(uniform(-5, 5))

-2.2594435836945594
1.6141879238412669
-3.1051125297083515
3.54713986705862
-3.229537214498789
-2.582324143313249
-4.428921711334887
-4.31360901873271


If you need an integer instead of a floating-point number (i.e., a number with a decimal point), you can wrap the call to `uniform()` in a call to `int()`. The cell below simulates rolling a [twenty-sided die](https://en.wikipedia.org/wiki/Dice#Standard_variations):

In [19]:
for i in range(8):
    print(int(uniform(20)+1))

10
11
2
8
19
3
13
7


> Exercise: Why do we need to add one to the result of the `uniform()` function in the example above?

### Normal distribution

A random number generator with a normal distribution (also called the Gaussian distribution or the "bell curve") produces values mostly clustered around a particular number. When generating a random number with a normal distribution, you specify the "center" of the distribution (i.e., the number that the values will cluster around) and the value for the standard deviation of the distribution, which controls the "spread" of numbers returned, above and below the center. (~70% of numbers will be within this range; ~95% will be within double the range.) Generate a random number with a normal distribution using the `normal()` function:

In [20]:
normal(0, 2)

1.6083206929402265

The first parameter is the center, and the second is the standard deviation. (These are often called *mu* and *sigma*, respectively.) Run it a number of times and you should see that the numbers generated mostly cluster around the center:

In [21]:
for i in range(8):
    print(normal(0, 2))

3.0004395612431627
-2.087416695104404
-1.7719859693761442
1.7819712426640846
-0.03851265120218291
-3.3707341220201643
-4.0835181213956036
-4.479536189776997


Normal distributions are very useful in computer-generated art because they allow us to introduce and control variation, keeping values mostly in the same range.

### Truncated normal distribution

You may have noticed that sometimes the random number generator for the normal distribution produces values well outside the standard deviation. That's not a bug—it's just how normal distributions work. (In any data set, there are sometimes extreme outliers.)

Nevertheless, in computer-generated art there's often a need to generate random numbers from a normal distribution that are *constrainted* to a particular range. (For example, you might want to generate a random color with a normal distribution, but never want your color values to fall outside of the range 0–255.) For this, you can use a *truncated normal distribution*. The following cell defines a function `t_normal()` to produce such values.

In [22]:
from scipy.stats import truncnorm
def t_normal(a, b, mu, sigma):
    tn = truncnorm((a-mu)/sigma, (b-mu)/sigma, loc=mu, scale=sigma)
    return tn.rvs(1)[0]

The first parameter is the low end of the range, the second parameter is the high end, the third parameter is the center and the fourth parameter is the standard deviation.

In [23]:
t_normal(0, 255, 10, 10)

19.440672471027277

### Choosing from a list with probabilities

The `choice()` function implements a kind of *weighted* random choice, where you can set the weights randomly. The code in the cell below picks one of `50`, `100` or `150` (the values between the first set of brackets). It will pick these with 15% probability, 80% probability, and 5% probability respectively (as specified by the values between the second set of brackets).

In [24]:
choice([50, 100, 150], p=[0.15, 0.8, 0.05])

100

Calling this function multiple times should result in the specified distribution:

In [25]:
for i in range(8):
    print(choice([50, 100, 150], p=[0.15, 0.8, 0.05]))

100
100
50
150
100
100
100
50


Without the `p` parameter, the `choice()` function defaults to a uniform distribution (i.e., no result is more likely than any other):

In [26]:
for i in range(8):
    print(choice([50, 100, 150]))

150
50
50
50
150
100
150
50


The `sample()` function (from Python's standard `random` module) takes a list of values and returns the specified number of values, randomly selected (using a uniform distribution) from the list.

In [27]:
from random import sample

In [28]:
sample([5, 10, 15, 20], 2)

[15, 10]

## Visualizing distributions with flat

The code in the cell below places circles on a page with the (x, y) coordinates of their centers determined by different random distributions. The small red circles are uniformly distributed; the medium-sized green circles are normally distributed and the large blue circles are in a truncated normal distribution (centered on the center of the page, with the same spread.)

In [29]:
d = document(100, 100, 'mm')
page = d.addpage()
uniform_fig = shape().nostroke().stroke(rgba(255, 0, 0, 240)).width(1).nofill()
normal_fig = shape().nostroke().stroke(rgba(64, 220, 64, 240)).width(1.5).nofill()
t_normal_fig = shape().nostroke().stroke(rgba(0, 128, 255, 240)).width(2).nofill()
mu = 50 # center of normal distributions
sigma = 20 # standard deviation of normal distributions
sample_n = 400 # number of "samples" (i.e. circles to draw for each distribution)
for i in range(sample_n):
    c = uniform_fig.circle(uniform(100), uniform(100), 1)
    page.place(c)
for i in range(sample_n):
    c = normal_fig.circle(normal(mu, sigma), normal(mu, sigma), 1.5)
    page.place(c)
for i in range(sample_n):
    c = t_normal_fig.circle(t_normal(20, 80, mu, sigma), t_normal(20, 80, mu, sigma), 2)
    page.place(c)
show(page)

> Exercise: Copy the visualization above, but rewrite it to visualize three normal distributions with different standard deviations.

## Random compositions

The following code uses random numbers to set the stroke color, fill color, and position of a rectangle on the page. (Run it multiple times to see the results.)

In [30]:
d = document(100, 100, 'mm')
page = d.addpage()
stroke_c = rgba(int(t_normal(0, 100, 50, 25)),
                int(t_normal(0, 100, 50, 25)),
                int(uniform(255)),
                255)
fill_c = rgba(int(uniform(255)),
              int(uniform(255)),
              int(uniform(255)),
              choice([40, 255], p=[0.9, 0.1]))
figure = shape().stroke(stroke_c).fill(fill_c).width(uniform(5))
rect = figure.rectangle(
    normal(25, 10), normal(25, 10),
    normal(50, 10), normal(20, 10))
page.place(rect)
show(page)

Play around with the parameters a bit until you understand how it works. Anywhere you see a call to `uniform()`, `normal()`, `t_normal()` or `choice()`, you can replace it with a call to another one of those same functions. (Note that colors values *always* have to be integers and always need to be between 0–255.)

The cell below uses a `for` loop to draw multiple rectangles:

In [31]:
d = document(100, 100, 'mm')
page = d.addpage()
for i in range(int(uniform(8, 24))):
    stroke_c = rgba(int(t_normal(0, 100, 50, 25)),
                    int(t_normal(0, 100, 50, 25)),
                    int(uniform(255)),
                    255)
    fill_c = rgba(int(uniform(255)),
                  int(uniform(255)),
                  int(uniform(255)),
                  choice([40, 255], p=[0.9, 0.1]))
    figure = shape().stroke(stroke_c).fill(fill_c).width(uniform(5))
    rect = figure.rectangle(normal(25, 10), normal(25, 10),
                            normal(50, 10), normal(20, 10))
    page.place(rect)
show(page)

Here's another example that uses circles instead of rectangles:

In [32]:
d = document(100, 100, 'mm')
page = d.addpage()
for i in range(12):
    fill_c = rgba(0, 0, int(uniform(255)), choice([40, 80], p=[0.5, 0.5]))
    figure = shape().nostroke().fill(fill_c).width(np.random.uniform(5))
    rect = figure.circle(normal(50, 10), normal(50, 10), t_normal(0, 100, 25, 15))
    page.place(rect)
show(page)

> Exercise: Write code to generate random compositions by copying one of the cells above and modifying the parameters. Also try adding in new shapes or combining different kinds of shapes.

> Exercise: Can you figure out how to make the "page" bigger? How do you need to adjust the parameters in the random number-generating functions to compensate?

## Saving to disk

If you make something with Flat that you really like, you can save it to disk for later use. You can do this in a number of ways, outputting to a number of formats. The `page` object's `.svg()` function returns a string of SVG:

In [33]:
page.svg()

b'<?xml version="1.0" encoding="UTF-8"?>\n<!-- Flat -->\n<svg version="1.1" xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" width="283.4646pt" height="283.4646pt">\n<title>Untitled</title>\n<circle cx="146.3691" cy="151.2731" r="18.6335" fill="rgba(0,0,168,0.3137)" />\n<circle cx="185.6457" cy="151.1557" r="13.2842" fill="rgba(0,0,86,0.1569)" />\n<circle cx="166.9037" cy="104.9333" r="111.1966" fill="rgba(0,0,108,0.1569)" />\n<circle cx="150.2185" cy="147.5948" r="53.9922" fill="rgba(0,0,190,0.1569)" />\n<circle cx="143.5755" cy="145.9254" r="152.645" fill="rgba(0,0,188,0.1569)" />\n<circle cx="154.6803" cy="191.1687" r="24.7348" fill="rgba(0,0,75,0.1569)" />\n<circle cx="138.1525" cy="163.8838" r="119.3536" fill="rgba(0,0,186,0.3137)" />\n<circle cx="98.5017" cy="137.2194" r="87.3363" fill="rgba(0,0,128,0.1569)" />\n<circle cx="110.32" cy="141.0821" r="65.4038" fill="rgba(0,0,188,0.1569)" />\n<circle cx="142.0885" cy="120.9609" r="83.1349" fill="rgba(0,0,

Which you can then write to disk using the following code:

In [34]:
with open("test.svg", "wb") as fh:
    fh.write(page.svg())

You can open this SVG file with most vector editors (Inkscape, Illustrator, etc.), though note that if you use `rgba` colors, the file might not work in some editors.

You can also save as an image using the `page` object's `.image()` method. For example, to save as a PNG:

In [35]:
with open("test.png", "wb") as fh:
    fh.write(page.image(ppi=72, kind="rgba").png())

Adjust the DPI with the `ppi` parameter. Note that Flat's PNG rasterizer is very slow, so preview at lower DPI before waiting for a higher-resolution render.

## Polylines

Okay, so, random numbers are good! But how does this help us model pen strokes? Good question. We're getting there! The next step in our asemic journey is to create a *series of lines* linking a given list of points (specified as (x, y) coordinates). The name for this shape—a series of connected lines—is a *polyline*.

The first step in making a polyline is to generate some points. The following cell creates a Python list with randomly-generated points:

In [36]:
pts = []
for i in range(10):
    pts.append([uniform(100), uniform(100)])

Here's what the points look like. This is a data structure called a *list of lists* (i.e., a list that has other lists as its elements), which for various reasons is a convenient way to store polylines and polygons.

In [37]:
pts

[[40.85284132792402, 46.32270774379332],
 [9.215160615198116, 54.82714605394162],
 [57.239306635139535, 44.36374664216965],
 [88.82777533318158, 1.1572041281684733],
 [94.91889954723077, 89.6513666116052],
 [27.41757733146227, 43.37195843736972],
 [62.92342419892663, 38.01448607369602],
 [15.666243476108164, 8.610865306226046],
 [59.396711293662115, 29.95914949559881],
 [95.2502900035431, 27.259000513968402]]

And here's some code to draw the points on a page, using a circle to mark each point.

In [38]:
d = document(100, 100, 'mm')
page = d.addpage()
figure = shape().stroke(rgba(0, 0, 0, 255)).width(1)
for item in pts: # loop through the pts list, assigning each point to variable item
    c = figure.circle(item[0], item[1], 2)
    page.place(c)
show(page)

The flat library has a built-in shape type, `.polyline()`, which draws lines connecting points. But the function wants the points as a list of points with the `x` and `y` coordinates directly following one another:

In [39]:
d = document(100, 100, 'mm')
page = d.addpage()
figure = shape().stroke(rgba(0, 0, 0, 255)).width(1)
lines = figure.polyline([20, 10, 40, 75, 90, 85, 15, 15])
page.place(lines)
show(page)

To get around this requirement of the library, the function below takes a list of lists and flattens it into a list (with the `x` and `y` components of the coordinates appropriately interleaved).

In [40]:
def flatten(t):
    from itertools import chain
    return list(chain(*t))

The code in the cell below draws a polyline from our randomly-generated points and draws circles where each point lies:

In [41]:
d = document(100, 100, 'mm')
page = d.addpage()
figure = shape().stroke(rgba(0, 0, 0, 255)).width(1)
lines = figure.polyline(flatten(pts))
page.place(lines)
for item in pts:
    c = figure.circle(item[0], item[1], 2)
    page.place(c)
show(page)

Combining all of this in the same cell, and getting rid of the circles on the points, gives us some code that generates designs that *almost* look like alien glyphs:

In [42]:
pts = []
for i in range(10):
    pts.append([uniform(100), uniform(100)])

d = document(100, 100, 'mm')
page = d.addpage()
figure = shape().stroke(rgba(0, 0, 0, 255)).width(1)
lines = figure.polyline(flatten(pts))
page.place(lines)
show(page)

## Polyline objects

The `bezmerizing` library comes with a `Polyline` object that represents a polyline. It has a number of useful features that we'll get into below. To make a `Polyline` object, pass in a 2d list of points to the constructor like so:

In [43]:
pts = [[10, 10], [20, 20], [10, 30], [50, 65], [65, 25]]
pline = Polyline(pts)

To draw a `Polyline` object, pass it to Flat's `polyline` function. (Note that you don't need to use `flatten()` when working with `Polyline` objects!)

In [44]:
d = document(80, 80, 'mm')
page = d.addpage()
figure = shape().stroke(rgba(0, 0, 0, 255)).width(1)
lines = figure.polyline(pline)
page.place(lines)
show(page)

A `Polyline` object can also be used to draw a polygon with Flat's `.polygon()` method. (Essentially all this does is close the path.)

In [45]:
d = document(80, 80, 'mm')
page = d.addpage()
figure = shape().stroke(rgba(0, 0, 0, 255)).fill(rgba(160, 0, 160, 255)).width(1)
lines = figure.polygon(pline)
page.place(lines)
show(page)

You can examine a `Polyline`'s contents by evaluating it:

In [46]:
pline

Polyline([[10.0000, 10.0000], [20.0000, 20.0000], [10.0000, 30.0000], [50.0000, 65.0000], [65.0000, 25.0000]])

And access the vertices directly (as a Numpy array) with the `.vertices` attribute:

In [47]:
pline.vertices

array([[10, 10],
       [20, 20],
       [10, 30],
       [50, 65],
       [65, 25]])

A `Polyline` object has `.translate()`, `.rotate()` and `.scale()` methods that operate similar to their equivalents in Processing. These methods apply the transformation to the vertices of the `Polyline` and return a new `Polyline` with the transformed vertices. For example:

In [48]:
pline.scale(2) # returns a new Polyline twice as big

Polyline([[20.0000, 20.0000], [40.0000, 40.0000], [20.0000, 60.0000], [100.0000, 130.0000], [130.0000, 50.0000]])

In [49]:
pline.translate(10, -5) # returns a new Polyline translated up and to the left

Polyline([[20.0000, 5.0000], [30.0000, 15.0000], [20.0000, 25.0000], [60.0000, 60.0000], [75.0000, 20.0000]])

In [50]:
pline.rotate(np.pi*0.25) # rotates the polyline pi/4 radians

Polyline([[14.1421, 0.0000], [28.2843, 0.0000], [28.2843, 14.1421], [81.3173, 10.6066], [63.6396, -28.2843]])

This makes it possible to write a bit of code that produces polylines like so:

In [51]:
pts = []
for i in range(12):
    pts.append([normal(), normal()])
pline = Polyline(pts)

And then draw the same polyline in multiple ways and in multiple places.

In [52]:
d = document(100, 100, 'mm')
page = d.addpage()
figure = shape().stroke(rgba(0, 0, 0, 255)).width(1)
figure2 = shape().stroke(rgba(128, 0, 128, 255)).width(2)
figure3 = shape().stroke(rgba(0, 128, 128, 255)).width(1)
# ten times bigger, at 50, 50
page.place(
    figure.polyline(pline.scale(10).translate(50, 50)))
# rotated
page.place(
    figure2.polyline(pline.scale(20).rotate(np.pi*0.5).translate(25, 25)))
# many small copies
for i in range(10):
    page.place(figure3.polyline(
        pline.scale(uniform(2, 4)).translate(uniform(50, 100), uniform(50, 100))))
show(page)

If you have other transformations in mind, you can always manipulate a Polyline's `.vertices` array directly:

In [53]:
# generate polyline
pts = []
for i in range(12):
    pts.append([normal(), normal()])
pline = Polyline(pts)

# draw polyline many times on top of itself
d = document(100, 100, 'mm')
page = d.addpage()
figure = shape().stroke(rgba(0, 0, 0, 24)).width(1)
for i in range(100):
    # add a bit of noise to the coordinates on each loop
    pline.vertices += np.random.randn(2) * 0.1
    lines = figure.polyline(pline.scale(10).translate(50, 50))
    page.place(lines)
show(page)

## Polylines on a grid

Let's pursue this aesthetic a little bit further. Introducing further constraints to the randomness will result in designs that give the impression of conforming to an underlying system (and, let's hypothesize, thereby give an impression of being more like "writing"). The code below also uses random polylines, but constrains the polylines to begin and end on certain points in the grid.

In [54]:
d = document(100, 100, 'mm')
page = d.addpage()
figure = shape().stroke(rgba(0, 0, 0, 255)).width(4).join('round')
pts = []
for i in range(12):
    x = choice([0, 0.5, 1])
    y = choice([0, 0.5, 1])
    pts.append([x, y])
pline = Polyline(pts)
lines = figure.polyline(pline.scale(90).translate(5, 5))
page.place(lines)
show(page)

The key bit of code here is:

In [55]:
choice([0, 0.5, 1])

0.0

... which is a Python expression that returns one of `0`, `0.5`, and `1` (at random). Using this expression for both the `x` and `y` coordinate of each point on the polyline leads to a result wherein each polyline randomly connects one of nine points on a grid. The `.scale()` and `.translate()` methods scale this polyline up and move it to the appropriate place on the screen (i.e., scaled up to 90 units, moved 5 units from the side just to give it a little border).

The cell below defines a function, `make_char()`, that generates a series of points using a more general version of this expression. It adds four to twelve segments to the polyline (randomly determined).

In [56]:
def make_char():
    pts = []
    for i in range(int(t_normal(4, 12, 8, 2))):
        x = (choice([0, 0.5, 1]))
        y = (choice([0, 0.5, 1]))
        pts.append([x, y])
    return Polyline(pts)

The results of the `make_char()` function:

In [57]:
make_char()

Polyline([[0.0000, 1.0000], [1.0000, 0.0000], [0.5000, 1.0000], [1.0000, 0.0000]])

The following cell draws these "characters" on a grid, so we can see many variations at once:

In [58]:
size = 150 # change the size of the page
d = document(size, size, 'mm')
page = d.addpage()
figure = shape().stroke(rgba(0, 0, 0, 255)).width(2).join('round')
grid_n = 8 # change the number of elements in the grid on one side
grid_size = size / grid_n
for i in range(grid_n):
    for j in range(grid_n):
        char_pline = make_char()
        lines = figure.polyline(
            char_pline.scale(grid_size * 0.67).translate(3+i*grid_size, 3+j*grid_size))
        page.place(lines)
show(page)

Now we're getting somewhere. When your sci-fi author friend texts you and says "Hey, I need something that looks like a writing system for space aliens for the cover of my next book," you've got something all ready to go.

> Exercise: Can you make these polylines fall on a 4x4 grid instead of a 3x3 grid? Or a 5x5 grid?

> Exercise: Add some randomness to each point's placement, so not all points fall directly on the grid.

> Exercise: Change the aspect ratio of the "characters"—make them wider than they are tall, or taller than they are wide.

## Interpolating curves

Due to certain properties of human physiology, handwriting hardly ever consists of exactly straight lines. To simulate the curved lines of conventional handwriting, we need to find a way to change our polylines into *curves*. One way of doing this is a [Catmull-Rom spline](https://en.wikipedia.org/wiki/Centripetal_Catmull%E2%80%93Rom_spline), which produces a sequence of [Bézier curves](https://pomax.github.io/bezierinfo/) that pass through a given set of points. Bezmerizing `Polyline` objects have a function `smooth_path()` that uses Catmull-Rom to produce a path (i.e., a sequence of drawing instructions) that can draw such a curve given a set of points.

The following cell generates some points with a normal distribution:

In [59]:
pts = []
for i in range(10):
    pts.append([normal(50, 25), normal(50, 25)])
pline = Polyline(pts)

And then this cell calls `smooth_point_path()` on the points, returning a list of curves:

In [60]:
curve_path = pline.smooth_path()

The object returned from `.smooth_path()` is a Bezmerizing `Path` object, that contains a series of Flat "path commands" to draw the curve:

In [61]:
curve_path

Path([moveto(x=43.3678, y=110.4152), curveto(x1=38.5817, y1=102.1854, x2=3.7847, y2=39.4416, x=6.1026, y=26.5831), moveto(x=6.1026, y=26.5831), curveto(x1=8.4204, y1=13.7247, x2=40.1116, y2=27.2713, x=57.2747, y=33.2645), moveto(x=57.2747, y=33.2645), curveto(x1=74.4378, y1=39.2576, x2=115.7525, y2=59.0433, x=109.0812, y=62.5419), moveto(x=109.0812, y=62.5419), curveto(x1=102.4099, y1=66.0406, x2=33.8272, y2=58.2024, x=17.2469, y=54.2563), moveto(x=17.2469, y=54.2563), curveto(x1=0.6667, y1=50.3103, x2=4.4780, y2=34.1277, x=9.5999, y=38.8659), moveto(x=9.5999, y=38.8659), curveto(x1=14.7219, y1=43.6040, x2=36.9422, y2=80.0125, x=47.9786, y=82.6853), moveto(x=47.9786, y=82.6853), curveto(x1=59.0149, y1=85.3581, x2=76.7506, y2=65.0242, x=75.8179, y=54.9025)])

Like `Polyline` objects, `Path`s can be translated, rotated, scaled, etc.

For our purpose, the exact nature of Bézier curves and the Flat library's path commands are not consequential. Suffice it to say that you can pass the result of `.smooth_path()` to the `.path()` method of a flat `shape` object. The cell below draws both the points and the curve passing through those points:

In [62]:
d = document(100, 100, 'mm')
page = d.addpage()
line_figure = shape().stroke(rgba(0, 0, 255, 128)).width(1)
curve_figure = shape().stroke(rgba(255, 0, 0, 255)).width(4)
lines = line_figure.polyline(pline)
curve = curve_figure.path(curve_path)
page.place(lines)
page.place(curve)
show(page)

You'll notice in the above example that the Catmull-Rom spline algorithm "eats" the first and last points of the polyline. (It uses those points to determine the position of the control points of the first and last curves of the spline.) Keep this in mind when you're using `.smooth_path()` and related functions—your procedure to generate glyphs needs to generate two extra points!

The following example copies the grid of alien letterforms above, but uses `.smooth_path()` to convert the points to curves before drawing. The results are much more "organic":

In [63]:
size = 100
d = document(size, size, 'mm')
page = d.addpage()
figure = shape().stroke(rgba(0, 0, 0, 255)).width(2).join('round')
grid_n = 8
grid_size = size / grid_n
for i in range(grid_n):
    for j in range(grid_n):
        char_pline = make_char()
        char_path = char_pline.smooth_path()
        curve = figure.path(
            char_path.scale(grid_size*0.67).translate(3+i*grid_size, 3+j*grid_size))
        page.place(curve)
show(page)

## Scribbling in a line

Writing in certain alphabets proceeds in a linear direction across the page with connected letters; English cursive does this in particular. To simulate this, I wrote a function that generates a polyline with random points evenly spaced along a line horizontally, with the y-coordinate of each point set according to a random number selected from a normal distribution centered on zero.

In [64]:
def make_scribble(width, height, steps, stddev=0):
    pts = []
    for i in range(steps):
        x = ((width / steps) * i) + normal(0, stddev)
        y = normal(0, height)
        pts.append([x, y])
    return Polyline(pts)

Here's what it looks like. (The green line is the original polyline; the black line is the smoothed curve.)

In [65]:
size = 200
d = document(200, 50, 'mm')
page = d.addpage()
line_figure = shape().stroke(rgba(0, 255, 0, 220)).width(2)
curve_figure = shape().stroke(rgba(0, 0, 0, 220)).width(2)
scribble_poly = make_scribble(width=180,
                    height=5,
                    steps=25,
                    stddev=0)
lines = line_figure.polyline(scribble_poly.translate(10, 25))
curve = curve_figure.path(scribble_poly.smooth_path().translate(10, 25))
page.place(curve) # comment to hide curves
page.place(lines) # comment to hide non-smooth lines
show(page)

The function also has a `stddev` parameter that allows you to adjust the standard deviation of a normally-distributed random number that adjusts the spacing of each point along the x-axis. This allows for the possibility of letters looping back on themselves:

In [66]:
size = 200
d = document(200, 50, 'mm')
page = d.addpage()
line_figure = shape().stroke(rgba(0, 255, 0, 255)).width(1)
scribble_poly = make_scribble(width=180,
                    height=5,
                    steps=100,
                    stddev=3)
curve = curve_figure.path(scribble_poly.smooth_path().translate(10, 25))
page.place(curve)
show(page)

The following example displays multiple "lines" of these scribbles, moving down the page, using a `for` loop:

In [67]:
size = 200
d = document(size, size, 'mm')
page = d.addpage()
line_figure = shape().stroke(rgba(0, 255, 0, 255)).width(1)
curve_figure = shape().stroke(rgba(0, 0, 0, 255)).width(2)
row_n = 16
for i in range(row_n):
    scribble_poly = make_scribble(
                        width=size-20,
                        height=(size/row_n)*0.25,
                        steps=int(uniform(25, 250)),
                        stddev=uniform(2))
    scribble_poly_tr = scribble_poly.translate(
        10, 10 + (i*size/(row_n+1))) # row_n + 1 to leave some space at the bottom
    lines = line_figure.polyline(scribble_poly_tr)
    curve = curve_figure.path(scribble_poly_tr.smooth_path())
    page.place(curve)
    #page.place(lines) # uncomment to see original lines
show(page)

> Exercise: Use the `i` parameter of the loop to control some other aspect of the line generation. (For example, make it so the number of steps increases as the scribbles move down the page.)

> Exercise: Introduce randomly-placed crosses (like the cross in `t`) and dots (as in `i`).

## Weighted strokes

The strokes produced by the flat library have a uniform width and texture, which makes the resulting artifacts resemble drawings made with a very precise pen. The problem of how to simulate how different writing implements make marks on a page ("brushes" in a tool like PhotoShop) is an open area of research in computer graphics research. ([A recent review](https://onlinelibrary.wiley.com/doi/full/10.1002/cav.1435).)

The Bezmerizing library implements a simple technique for simulating an ink brush or other kinds of pens, which works by drawing a polygon along the tangents of the curves composing the stroke. The `Polyline` object has a `.fancy_curve()` method that performs the technique. To demonstrate, let's create a `Polyline` object with a set of points:

In [68]:
pline = Polyline([[10, 10], [15, 15], [15, 85], [50, 50], [85, 85], [85, 15], [90, 10]])

... and then create the polygon around the curve with `.fancy_curve()`. The `samples_per` parameter controls the resolution of the polygon (lower values are faster to calculate but chunkier) and the `thicknesses` parameter controls the thickness of the polygon along the path.

In [69]:
curve_poly = pline.fancy_curve(samples_per=24, thicknesses=[0, 2, 10, 2, 8, 2])

Here's an example that draws the original polyline, the Catmull-Rom curve created from the polyline, and the simulated brushstroke, just to show you how it works:

In [70]:
d = document(100, 100, 'mm')
page = d.addpage()
curve_figure = shape().nostroke().fill(rgba(0, 0, 0, 240))
smooth_figure = shape().stroke(rgba(0, 255, 0, 255))
poly_figure = shape().stroke(rgba(255, 0, 0, 255))
smooth_path = pline.smooth_path()
fancy_poly = curve_figure.polygon(curve_poly)
path = smooth_figure.path(smooth_path)
poly = poly_figure.polyline(pline)
page.place(fancy_poly)
page.place(path)
page.place(poly)
show(page)

Here's the "alien characters" example above, except using `fancy_curve()` to generate polygons for each stroke. (Try playing around with the `thickness` parameter.)

In [71]:
size = 150
d = document(size, size, 'mm')
page = d.addpage()
figure = shape().nostroke().fill(rgba(0, 0, 0, 255))
grid_n = 8
grid_size = size / grid_n
for i in range(grid_n):
    for j in range(grid_n):
        char_pline = make_char().scale(grid_size * 0.67).translate(3 + i * grid_size, 3 + j * grid_size)
        poly = char_pline.fancy_curve(samples_per=12, thicknesses=[3, 1, 1, 2], tightness=-0.25)
        poly = figure.polygon(poly)
        page.place(poly)
show(page)

## Connecting polylines

`Polyline` objects can be added together, creating a new polyline that combines their points:

In [72]:
a = Polyline([[10, 10], [20, 20]])
b = Polyline([[25, 35], [20, 25]])
a + b

Polyline([[10.0000, 10.0000], [20.0000, 20.0000], [25.0000, 35.0000], [20.0000, 25.0000]])

We can use this feature to combine two glyphs from the `make_char()` function, translating the second glyph to be to the right of the first glyph.

In [73]:
composite = make_char() + make_char().translate(1.5, 0)
composite = composite.scale(30).translate(10, 10) # scale it up so we can actually see it!

Here's what it looks like (original polyline in light purple, curve in cyan):

In [74]:
d = document(100, 50, 'mm')
page = d.addpage()
line_fig = shape().stroke(rgba(160, 80, 160, 255)).nofill()
curve_fig = shape().stroke(rgba(0, 128, 128, 255)).nofill()
page.place(line_fig.polyline(composite))
page.place(curve_fig.path(composite.smooth_path()))
show(page)

To make a whole row of these, just keep appending glyphs with increasing translation along the X axis:

In [75]:
line = make_char()
for i in range(1, 10):
    line += make_char().translate(i*1.5, 0)
line = line.scale(10).translate(10, 10)
d = document(200, 30, 'mm')
page = d.addpage()
line_fig = shape().stroke(rgba(160, 80, 160, 255)).nofill()
curve_fig = shape().stroke(rgba(0, 128, 128, 255)).nofill()
page.place(line_fig.polyline(line))
page.place(curve_fig.path(line.smooth_path()))
show(page)

The following example takes advantage of this technique in order to produce a composition of several lines. As a change of pace, let's work with a limited set of glyphs, instead of generating a new glyph every time we want to place one on the page:

In [76]:
chars = [make_char() for i in range(26)]

Working with a limited set of glyphs (in my experience) provides a bit of verisimilitude, since you expect to see repeated figures in handwriting. But you also have to be careful to add a bit of variation to the glyphs, because otherwise it can look a bit "robotic." (Though "robotic" might be exactly what you're going for. I don't know.)

The code below generates ten lines of these "cursive" (i.e., connected) forms:

In [77]:
d = document(150, 150, 'mm')
page = d.addpage()
line_fig = shape().stroke(rgba(128, 0, 128, 255)).nofill()
curve_fig = shape().stroke(rgba(0, 128, 128, 255)).nofill()
for row in range(10):
    line = choice(chars)
    for col in range(1, 10):
        # scale(normal(...)) adds a bit of size variation to the glyphs
        line += choice(chars).scale(normal(1, 0.25)).translate(col*1.5, 0)
    # translate the row to the right place on the screen and make it bigger
    line = line.translate(1, 1+row*1.5).scale(9)
    smooth = line.smooth_path()
    #page.place(line_fig.polyline(line)) # uncomment to see original polylines
    page.place(curve_fig.path(smooth))
show(page)